In [2]:
from Model_Training import *
from Cleaner import Cleaner
import plotly.express as px
import warnings
import pandas as pd
from EmpiricDistribution import EmpiricDistribution
from datetime import datetime, time
import pickle # Para guardar los modelos entrenados
from datetime import time 
import matplotlib.pyplot as plt
import numpy as np
import random
from sympy import Symbol
from datetime import time
import random
import simpy


warnings.filterwarnings('ignore')

buses_model = Buses_model()
pax_model = Pax_model()

#INTIAL SETUP
N_BUSES = 2
BUS_SAMPLES = 2000
PAX_SAMPLES = 10000

### Buses

In [3]:
#Uncomment this line if you dont have the time_models.pkl

#buses_model.generate_model()

In [4]:
#Uncomment this line if you dont have the bus_results_times.csv
#buses_model.generate_simulation(n_samples=BUS_SAMPLES)

In [5]:
#Show histograms of real data vs simulated data (to validate model)

#buses_model.show_results()

### PAX

In [6]:
#You can comment this if you already have the pax_time_arrivals.csv
pax_model.generate_model(n_buses=N_BUSES)
pax_model.generate_simulation(n_samples=PAX_SAMPLES)

### DISCRETE SIMULATION

In [10]:
def calcular_frecuencia_salida(df_original: pd.DataFrame) -> pd.DataFrame:
    
    # Definición de Segmentos
    TRAYECTOS = ['A2-GLAXO', 'GLAXO-DISNEY', 'DISNEY-A2']
    TIME_INTERVALS_LIST = [
        ('07:00', '09:30'), 
        ('13:30', '18:00'), 
        ('06:00', '07:00'), 
        ('09:30', '13:30'), 
        ('18:30', '22:15')
    ]
    

    if not isinstance(df_original['hora_salida'].iloc[0], time):
        df_original['hora_salida'] = pd.to_datetime(df_original['hora_salida'], format='%H:%M', errors='coerce').dt.time

            
    resultados_frecuencia = []
    
    # Iteración y Cálculo
    for trayecto in TRAYECTOS:
        for start_time_str, end_time_str in TIME_INTERVALS_LIST:
            
            # Definición de Intervalo y Duración
            time_start = time.fromisoformat(start_time_str)
            time_end = time.fromisoformat(end_time_str)
            
            # Calcular la duración del intervalo en horas
            dt_start = pd.to_datetime(start_time_str, format='%H:%M')
            dt_end = pd.to_datetime(end_time_str, format='%H:%M')
            
            duration_hours = (dt_end - dt_start).total_seconds() / 3600
            
            
            # Filtrado de Datos
            sub_trayecto = df_original[df_original['trayecto'] == trayecto].copy()
            
            df_segmento = sub_trayecto.loc[
                (sub_trayecto['hora_salida'] >= time_start) & 
                (sub_trayecto['hora_salida'] < time_end)
            ]
            
            # Cálculo de la Frecuencia
            
            # Contar el número total de salidas en este segmento, a través de todos los días
            total_salidas = len(df_segmento)
            
            # Contar el número de días únicos que tienen datos en el segmento
            total_dias = df_segmento['fecha'].nunique()
            
            # Frecuencia Total de Salidas por la duración del intervalo
            if total_salidas > 0 and total_dias > 0:
                frecuencia_salida_por_hora = total_salidas / (total_dias * duration_hours)
            else:
                frecuencia_salida_por_hora = 0.0
            
            
            # Almacenamiento de Resultados
            resultados_frecuencia.append({
                'Segmento': f"{trayecto}_{start_time_str.replace(':', '')}-{end_time_str.replace(':', '')}",
                'Trayecto': trayecto,
                'Intervalo': f"{start_time_str}-{end_time_str}",
                'Duracion_Horas': round(duration_hours, 2),
                'Total_Dias_Datos': total_dias,
                'Total_Salidas_Historicas': total_salidas,
                'Frecuencia_Salida_por_Hora': round(frecuencia_salida_por_hora, 2)
            })
            
    return pd.DataFrame(resultados_frecuencia)


df_clean = pd.read_csv('data_clean.csv')

# Ejecutar la función
df_frecuencias = calcular_frecuencia_salida(df_clean.copy())

# Mostrar el resultado
print("--- Frecuencia de Salida de Autobuses (Salidas por Hora) ---")
print(df_frecuencias)

--- Frecuencia de Salida de Autobuses (Salidas por Hora) ---
                  Segmento      Trayecto    Intervalo  Duracion_Horas  \
0       A2-GLAXO_0700-0930      A2-GLAXO  07:00-09:30            2.50   
1       A2-GLAXO_1330-1800      A2-GLAXO  13:30-18:00            4.50   
2       A2-GLAXO_0600-0700      A2-GLAXO  06:00-07:00            1.00   
3       A2-GLAXO_0930-1330      A2-GLAXO  09:30-13:30            4.00   
4       A2-GLAXO_1830-2215      A2-GLAXO  18:30-22:15            3.75   
5   GLAXO-DISNEY_0700-0930  GLAXO-DISNEY  07:00-09:30            2.50   
6   GLAXO-DISNEY_1330-1800  GLAXO-DISNEY  13:30-18:00            4.50   
7   GLAXO-DISNEY_0600-0700  GLAXO-DISNEY  06:00-07:00            1.00   
8   GLAXO-DISNEY_0930-1330  GLAXO-DISNEY  09:30-13:30            4.00   
9   GLAXO-DISNEY_1830-2215  GLAXO-DISNEY  18:30-22:15            3.75   
10     DISNEY-A2_0700-0930     DISNEY-A2  07:00-09:30            2.50   
11     DISNEY-A2_1330-1800     DISNEY-A2  13:30-18:00          